In [ ]:
import pandas as pd
import pandas as pd
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
path_of_data = 'data.log' 

In [ ]:
with open(path_of_data) as f:
    lines = f.readlines()

In [ ]:
data = []
index = 0
names = []
for i in lines:
    split = i.split(',')[3:]
    data.append({})
#     data[index]['name'] = split[0]
#     names.append(split[0])
    data[index]['missions'] = int(split[1])
    data[index]['failed_missions'] = int(split[2])
    data[index]['num_missions_voted_up_with_total_suspect_count'] =  [int(x) for x in split[3:9]]
    data[index]['num_missions_voted_down_with_total_suspect_count'] = [int(x) for x in split[9:15]]
    data[index]['spy'] = int(split[15][0])
    index = index + 1

In [ ]:
df = pd.DataFrame.from_dict(data)

df[['up_0', 'up_1','up_2', 'up_3','up_4', 'up_5']] = pd.DataFrame(df["num_missions_voted_up_with_total_suspect_count"].to_list(), columns=['up_0', 'up_1','up_2', 'up_3','up_4', 'up_5'])

df[['down_0', 'down_1','down_2', 'down_3','down_4', 'down_5']] = pd.DataFrame(df["num_missions_voted_down_with_total_suspect_count"].to_list(), columns=['down_0', 'down_1','down_2', 'down_3','down_4', 'down_5'])

df = df.drop('num_missions_voted_down_with_total_suspect_count', axis=1)
df = df.drop('num_missions_voted_up_with_total_suspect_count', axis=1)

result = df['spy']

df = df.drop('spy',axis = 1)

In [ ]:
df

In [ ]:
df = (df - df.min()) / df.max() - df.min() # normalize the data

In [ ]:
# create model
from keras.models import model_from_json

# Fit the model
model = Sequential()
model.add(Dense(512, input_dim=14, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(df, result, epochs=1000, batch_size=32)

scores = model.evaluate(df, result, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


In [ ]:
scores = model.evaluate(df, result, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# serialize model to JSON
model_json = model.to_json()
with open("resistance_nn_model_improved.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("resistance_nn_model_improved.h5")
print("Saved model to disk")